# 1.7 第一个应用：鸢尾花(Iris)分类

## 1.7.1 初识数据

In [34]:
# 调用load_iris函数来加载数据
from sklearn.datasets import load_iris
iris_dataset = load_iris()

In [35]:
# load_iris返回的iris对象是一个Bunch对象，与字典类似，里面包含键和值
print("Keys of iris_dataset: \n{}".format(iris_dataset.keys()))

Keys of iris_dataset: 
dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [36]:
# DESCR键对应的值是数据集的简要说明。下面给出说明的开头部分
print(iris_dataset['DESCR'][:440] + "\n...")

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica


...


In [37]:
# target_names键对应的值是一个字符串数组，里面包含的是我们要预测的花的品种
print("Target names: \n{}".format(iris_dataset['target_names']))

Target names: 
['setosa' 'versicolor' 'virginica']


In [38]:
# feature_names键对应的值是一个字符串列表，对每一个特征进行了说明: sepal - 花萼， petal - 花瓣
print("Feature names: \n{}".format(iris_dataset['feature_names']))

Feature names: 
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [39]:
# 数据包含在target和data字段中。
# data里面是花萼长度、花萼宽带、花瓣长度、花瓣宽带的测量数据，格式是NumPy数组
print("Type of data: {}".format(type(iris_dataset['data'])))

Type of data: <class 'numpy.ndarray'>


In [40]:
# data数组的每一行对应一朵花，列代表每朵花的四个测量数据
print("Shape of data: {}".format(iris_dataset['data'].shape))

Shape of data: (150, 4)


In [41]:
# 上面数组中包含150朵不同的花的测量数据
# 机器学习的个体叫作样本(sample)，其属性叫作特征(feature)，data数组的形状(shape)是样本数乘以特征数。
# 下面是前5个样本的特征数值
print("First 5 rows of data:\n{}".format(iris_dataset['data'][:5]))

First 5 rows of data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [42]:
# target数组包含的是测量过的每朵花的品种，也是一个NumPy数组
print("Type of target: {}".format(type(iris_dataset['target'])))

Type of target: <class 'numpy.ndarray'>


In [43]:
# target是一个一维数组，每朵花对应其中一个数据
print("Shape of target: {}".format(iris_dataset['target'].shape))

Shape of target: (150,)


In [44]:
# 品种被转换为从0到2的整数
print("Target:\n{}".format(iris_dataset['target']))

Target:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


上述数字的代表含义由`iris['target_names']`数组给出：
- 0: setosa 丝状
- 1: versicolor 杂色
- 2: virginica 维吉尼亚
（意大利语）

## 1.7.2 衡量模型是否成功：训练数据与测试数据

我们需要用新数据来评估模型的性能。新数据是指模型之前没有见过的数据，而我们有这些新数据的标签。

通常的做法是将收集好的带标签数据（这个鸢尾花数据集是150朵花的测量数据）分成两个部分：
- 一部分数据用于构建机器学习模型，叫作**训练数据(training data)**或**训练集(training set)**
- 其余的数据用来评估模型性能，叫作**测试数据(test data)**、**测试集(test set)**或**留出集(hold-out set)**

scikit-learn中的`train_test_split`函数可以打乱数据集并进行拆分，将75%的行数据及对应标签作为训练集，剩下25%的数据及其标签作为测试集。（训练集和测试集的分配比例可以是随意的，但使用25%的数据作为测试集是很好的经验法则）。

scikit-learn中的数据通常用大写的`X`表示，而标签用小写的`y`表示。这是受到了数学标准公式`f(x)=y`的启发，其中`X`是函数的输入，`y`是输出。

我们用大写的`X`是因为数据是一个二维数组（矩阵），用小写的`y`是因为目标是一个一维数组（向量）。

In [45]:
# 对数据调用train_test_split，并对输出结果采用如下命名方法：
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_dataset['data'], iris_dataset['target'], random_state=0)

In [48]:
# train_test_split函数的输出为X_train, X_test, y_train, y_test，它们都是NumPy数组。
# X_train包含75%的行数据，X_test包含剩下的25%
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (112, 4)
y_train shape: (112,)
X_test shape: (38, 4)
y_test shape: (38,)


## 1.7.3 要事第一：观察数据